## Step 2 Network Reconciliation
---
Use this notebook to setup a semi-automated reconciliation process using functions available in 'conflation_tools.py' and 'network_reconcile.py.' Unlike the other notebooks, this can't just be run block by block.

In general, you want to select one network to act as the base network (ground-truth) and add network data/attributes from the other networks.

These are the main functions in the conflation_tools module (type help(function_name) for a detailed description):
- match_nodes: finds node pairs between base and join network
- split_lines_create_points: Uses points from the join network to split links in the base network
- add_split_links: add the split links 

From network_filter
- add_ref_ids: adds new reference ids from teh nodes layer

The network_reconcile module contains functions for dealing with attribute data
- add_attributes: Uses a buffer and difference in heading method to determine link pairs from one network to another

The following three functions add attribute data back into the network and pre-processes it to match up with the desired impedance columns. These are custom network specific functions, so if adding a new network, will need to make another specific function.
- add_osm_attr
- add_here_attr
- add_abm_attr

Once finished reconciling, network can be exported for manual reconciling or it can be prepped for network routing in BikewaySim.

type "help(insert_name_of_function)" to get more information about what the function does.

## Import Modules

In [2]:
from pathlib import Path
import geopandas as gpd
import pandas as pd

from conflation_tools import *
from network_filter import *
from network_reconcile import *

# Network Reconciliation for NCST Project Bike Portion
---
This workflow was used in reconciling the network used in the second part of the NCST project.

In the bike portion, the OSM network serves as the base network, and the HERE road network is used to add additional road attributes on speed, the number of lanes, etc. The HERE attributes will only be added to the OSM road layer to minimize incorrect matches. (NOTE: attribute matches will need to be QA/QC'd manually, this function just serves to populate a base network with the most likely match)
   
In addition, a non-network geojson file of the Atlanta Regional Comission's Regional Bikeway Inventory 2022 will be used to add additional info to the network.

Workflow:
- Import OSM road and bike layers
- Connect OSM road and bike layers via the dead end links in the bike layer
- Add attributes from HERE data and supplemental data on cycling infrastructure from the ARC 2022 Bicycle Facility Inventory
- Resolve and simpilify attribute data for impedance calculation in the next notebook
- Export

## Import OSM road and bike layers

In [ ]:
# project directory
filepath = Path.home() / 'Documents/NewBikewaySimData/networks'

#import road layer
road_links = gpd.read_file(filepath / 'filtered.gpkg',layer='osm_links_road')
road_nodes = gpd.read_file(filepath / 'filtered.gpkg',layer='osm_nodes_road')

#import bike layer
bike_links = gpd.read_file(filepath / 'filtered.gpkg',layer='osm_links_bike')
bike_nodes = gpd.read_file(filepath / 'filtered.gpkg',layer='osm_nodes_bike')

## Connect OSM road and bike layers via the dead end links in the bike layer
In this cell, the OSM road and bike layers are reconnected to form one network. To do this, dead end nodes on the bike layer are used to split osm final and create new nodes if no node is near.

In [ ]:
#get node count to find dead ends
bike_nodes['num_links'] = bike_nodes['osm_N'].map(pd.concat([bike_links['osm_A'],bike_links['osm_B']],ignore_index=True).value_counts())
dead_ends = bike_nodes[bike_nodes['num_links']==1]

#remove dead ends already connected to road network
dead_ends = dead_ends[-dead_ends['osm_N'].isin(road_nodes['osm_N'])]

#split osm road links
split_lines, split_points, unmatched_join_nodes = split_lines_create_points(dead_ends, 'osm', road_links, 'osm', 40)

#add bike nodes to road nodes
road_nodes = pd.concat([road_nodes,split_points,bike_nodes],ignore_index=True).drop_duplicates()

#add ref ids to split links and then add to road links (replaces the links that were split)
split_lines = add_ref_ids(split_lines,road_nodes,'osm')
road_links= add_split_links(road_links,split_lines,'osm')

#add bike links to road links
road_links = pd.concat([road_links,bike_links],ignore_index=True)

## Add supplemental attributes to OSM network from HERE and the ARC bicycle inventory

In [ ]:
help(add_attributes)

In [ ]:
#add attribute data back to OSM
osm = add_osm_attr(road_links,working_dir / f"{studyarea_name}/osm.pkl")

#import here data and attribute data
here = gpd.read_file(working_dir / "{studyarea_name}/filtered.gpkg",layer='here_links_road')
here = add_here_attr(here,working_dir / f"{studyarea_name}/here.pkl")

#import arc bicycle inventory data
arc_bike = gpd.read_file(working_dir / 'Data/ARC/Regional_Bikeway_Inventory_2022.geojson').to_crs('epsg:2240')

#add here road attributes to osm
osm = add_attributes(osm, here, 'here', 25 0.9,, dissolve=True)

#add arc bike attributes to osm
final = add_attributes(road_links, arc_bike, 'arc_bike', 50, 0.9, dissolve=False)

### Reconcile attributes
Before exporting to file, we want to simplify the attributes so that an impedance function can be easily applied. In this code block, the lane attributes are 

In [ ]:
networks = ['osm','here','arc']
cols = ['bl','pbl','mu','<25mph','25-30mph','>30mph','1lpd','2-3lpd','>4lpd']

for col in cols:
    final[col] = 0
    final[col+'_check'] = final[[ x for x in final.columns.tolist() if col in x]].sum(axis=1)

#bike facil (go with osm values for multi-use)
final.loc[final['osm_mu'] == 1, 'mu'] = 1
final.loc[final['pbl_check'] > 0,'pbl'] = 1
final.loc[final['bl_check'] > 0,'bl'] = 1

#get bike facilites and export
final['bike_facil'] = None
final.loc[final['mu'] == 1, 'bike_facil'] = 'Multi-Use Path'
final.loc[final['bl'] == 1, 'bike_facil'] = 'Bike Lane'
final.loc[final['pbl'] == 1, 'bike_facil'] = 'Protected Bike Lane'

#speed (go with here values)
final.loc[final['here_<25mph'] == 1, '<25mph'] = 1
final.loc[final['here_25-30mph'] == 1, '25-30mph'] = 1
final.loc[final['here_>30mph'] == 1, '>30mph'] = 1

#lanes (go with here values)
final.loc[final['here_1lpd'] == 1, '1lpd'] = 1
final.loc[final['here_2-3lpd'] == 1, '2-3lpd'] = 1
final.loc[final['here_>4lpd'] == 1, '>4lpd'] = 1

#make all other 0 if mu
new_cols = ['bl','pbl','<25mph','25-30mph','>30mph','1lpd','2-3lpd','>4lpd']
final.loc[final['mu']==1,new_cols] = 0

#have pbl over bl
final.loc[(final['bl']==1) & (final['pbl']==1),'bl'] = 0

#links with no attributes (use highway tag to impute)
no_values = final[cols].sum(axis=1) == 0
final.loc[no_values & (final['highway']=='residential'),'25-30mph'] = 1
final.loc[no_values & (final['highway']=='residential'),'1lpd'] = 1

final.loc[no_values & (final['highway']=='service'),'<25mph'] = 1
final.loc[no_values & (final['highway']=='service'),'1lpd'] = 1

#make this more advanced later
others = ['secondary','trunk','trunk_link','tertiary','tertiary_link','primary','secondary_link','primary_link']
final.loc[no_values & (final['highway'].isin(others)),'25-30mph'] = 1
final.loc[no_values & (final['highway'].isin(others)),'2-3lpd'] = 1

final = final[['osm_A','osm_B','osm_A_B','name','highway','oneway','ST_NAME','FUNC_CLASS','DIR_TRAVEL','bike_facil','geometry']+cols]

## Export for next step

In [ ]:
final.to_file(working_dir / studyarea_name / 'reconciled_network.gpkg',layer='links')
road_nodes.to_file(working_dir / studyarea_name / 'reconciled_network.gpkg',layer='nodes')

## Look at where the bike facilities were added

In [ ]:
final.explore('bike_facil')

# NCST Project Workflow for Transit Part
---
This workflow was used in reconciling the network used in the second part of the NCST project.

- Import OSM road and bike layers
- Find dead end bike nodes that aren't already present in the road layer
- Split road links by these dead ends and add new split links to road layer
- Add OSM attributes back to the road network
- Export
- Create LTS version by taking out stressful links
- Export

In [3]:
# project directory
filepath = Path.home() / 'Documents/TransitSimData/networks'

#import road layer
road_links = gpd.read_file(filepath / 'filtered.gpkg',layer='osm_links_road')
road_nodes = gpd.read_file(filepath / 'filtered.gpkg',layer='osm_nodes_road')

#import bike layer
bike_links = gpd.read_file(filepath / 'filtered.gpkg',layer='osm_links_bike')
bike_nodes = gpd.read_file(filepath / 'filtered.gpkg',layer='osm_nodes_bike')

In this cell, the OSM road and bike layers are reconnected to form one network. To do this, dead end nodes on the bike layer are used to split osm final and create new nodes if no node is near.

In [ ]:
#get node count to find dead ends
bike_nodes['num_links'] = bike_nodes['osm_N'].map(pd.concat([bike_links['osm_A'],bike_links['osm_B']],ignore_index=True).value_counts())
dead_ends = bike_nodes[bike_nodes['num_links']==1]

#remove dead ends already connected to road network
dead_ends = dead_ends[-dead_ends['osm_N'].isin(road_nodes['osm_N'])]

#split osm road links
split_lines, split_points, unmatched_join_nodes = split_lines_create_points(dead_ends, 'osm', road_links, 'osm', 40)

#add bike nodes to road nodes
road_nodes = pd.concat([road_nodes,split_points,bike_nodes],ignore_index=True).drop_duplicates()

#add ref ids to split links and then add to road links (replaces the links that were split)
split_lines = add_ref_ids(split_lines,road_nodes,'osm')
road_links= add_split_links(road_links,split_lines,'osm')

#add bike links to road links
road_links= pd.concat([road_links,bike_links],ignore_index=True)

In [5]:
#add attributes back
osm = add_osm_attr(road_links, filepath / 'osm.pkl')

#export
osm.to_file(filepath / 'reconciled_network.gpkg',layer='links')
road_nodes.to_file(filepath / 'reconciled_network.gpkg',layer='nodes')

In [4]:
#add attributes back
osm = add_osm_attr(road_links, filepath / 'osm.pkl')

#drop stressful links unless they have bike infra
stressful = osm['highway'].isin(['primary','primary_link','secondary','secondary_link','tertiary_link','trunk','trunk_link'])
bike_infra = osm[['osm_pbl','osm_mu','osm_bl']].sum(axis=1) > 0
new_osm = osm[-stressful | bike_infra]

#retrieve nodes
new_nodes = road_nodes[road_nodes['osm_N'].isin(new_osm['osm_A'].append(new_osm['osm_B']).drop_duplicates())]

#export
new_osm.to_file(filepath / 'lowstress_network.gpkg',layer='links')
new_nodes.to_file(filepath / 'lowstress_network.gpkg',layer='nodes')